In [ ]:
import pandas as pd
from custom.utils import key2
import get_param_in_cwd
import numpy as np
import os

if not os.path.exists('features'):
        os.makedirs('features')

model_root = get_param_in_cwd('model_root', 'models')
model_name = get_param_in_cwd('model_name', 'densenet121')

train_log = pd.read_csv(os.path.join(model_root, model_name, 'viz/BST_TRAIN_RESULTS.txt'), sep='\t',
                        names=['fname', 'prob', 'pred', 'gt'])

val_log = pd.read_csv(os.path.join(model_root, model_name, 'viz/BST_VAL_RESULTS.txt'), sep='\t',
                      names=['fname', 'prob', 'pred', 'gt'])
log = pd.concat([train_log, val_log], axis=0)
log['prob'] = list(map(lambda x: x[0] if x[1] == 1 else 1-x[0], np.array(log[['prob', 'pred']])))
log['prob'] = log['prob'].round(decimals=2)
log[['group']] = log[['fname']].applymap(lambda x: os.path.basename(x).split('_')[0])
log.head()

In [ ]:
histo_results = key2.key2histogram(log, group_column='group',histo_columns='prob', norm=True)
histo_results.to_csv('features/histogram.csv', header=True, index=False)
histo_results

In [ ]:
tfidf_results = key2.key2tfidf(log, group_column='group',corpus_columns='prob')
tfidf_results.to_csv('features/tfidf.csv', header=True, index=False)
tfidf_results

In [ ]:
import pandas as pd

results = pd.merge(histo_results, tfidf_results, on='ID', how='inner', suffixes=['_histo', '_tfidf'])
results.to_csv('features/histo_tfidf.csv', header=True, index=False)
results